# Анализ эксперимента neural_controller-v1

## Постановка эксперимента

**Задача**: стабилизация лазера с помощью нейросетевого контроллера, обученного методом Twin Delayed DDPG (TD3).

**Окружение**: `NeuralPIDDeltaEnv` — агент управляет фазовращателем через последовательный порт (COM3). 
- **Наблюдение**: `[error, error_prev, error_prev_prev]` — текущая и две предыдущие ошибки (нормализованы).
- **Действие**: нормализованное приращение управляющего сигнала `delta_norm ∈ [-1, 1]`, масштабируемое до `delta ∈ [-30, 30]`.
- **Награда**: `-|error|` — минимизация абсолютной ошибки.
- **Уставка (setpoint)**: 1200 (относительно `process_variable_max = 10230`).
- **Управляющий сигнал**: `control_output ∈ [0, 4095]`, начальное значение при ресете — 2000.

**Алгоритм**: TD3, MLP 256×256, `gamma = 0.99`, `tau = 0.005`, `policy_freq = 2`.

**Сбор данных**: асинхронный коллектор, синхронизация весов каждые 10 шагов.

**Исследование (exploration)**: PID-контроллер (`kp=3.5, ki=11.0, kd=0.002`) в течение первых 20 000 шагов окружения.

**Обучение**: начинается после 5 000 шагов буфера, буфер на 100 000 переходов, батч 256.

**Длительность**: 2026-02-13 14:59 → 16:13 (~1 ч 14 мин), прерван пользователем.

In [ ]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.paths import get_experiment_dir

In [ ]:
EXPERIMENT_NAME = "neural_controller-v1"
EXPERIMENT_DATE = "2026-02-13"
EXPERIMENT_TIME = "14-59-23"

EXPERIMENT_DIR_PATH = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

config = load_config(EXPERIMENT_DIR_PATH / "config.yaml")
print(f"Эксперимент: {config.experiment_name}")
print(f"Окружение: {config.env.name}")
print(f"Алгоритм: {config.algorithm.type}")
print(f"Exploration steps: {config.exploration.steps}")
print(f"Train start step: {config.training.train_start_step}")

## Загрузка и парсинг данных

In [ ]:
def load_jsonl(path: Path, source: str | None = None, event: str | None = None) -> pd.DataFrame:
    rows = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
            except json.JSONDecodeError:
                continue
            if source and record.get("source") != source:
                continue
            if event and record.get("event") != event:
                continue
            rows.append(record)
    return pd.DataFrame(rows)

In [ ]:
env_df = load_jsonl(EXPERIMENT_DIR_PATH / "env.jsonl", source="env", event="step")
env_df = env_df.reset_index(drop=True)
env_df['global_step'] = env_df.index + 1  # абсолютный номер шага (step сбрасывается при reset)
print(f"Шаги окружения: {len(env_df)} записей")
print(f"Диапазон global_step: {env_df['global_step'].min()} — {env_df['global_step'].max()}")
env_df.tail()

In [ ]:
train_df = load_jsonl(EXPERIMENT_DIR_PATH / "train.jsonl", source="train", event="step")
print(f"Шаги обучения: {len(train_df)} записей")
print(f"Диапазон шагов: {train_df['step'].min()} — {train_df['step'].max()}")
train_df.head()

In [ ]:
exploration_steps = config.exploration.steps
print(f"Exploration steps: {exploration_steps}")

## Анализ логов окружения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['error'], alpha=0.8, linewidth=0.5, label='Error')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Ошибка регулирования')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['reward'], alpha=0.8, linewidth=0.5, color='tab:green', label='Reward')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Награда')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Reward')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['process_variable'], 'b-', alpha=0.7, linewidth=0.5, label='Process Variable')
plt.plot(env_df['global_step'], env_df['setpoint'], 'r--', linewidth=1.5, label='Setpoint')
plt.axvline(x=exploration_steps, color='orange', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['control_output'], 'g-', alpha=0.7, linewidth=0.5, label='Control Output')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Управляющий сигнал')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Control Output')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['delta_norm'], alpha=0.7, linewidth=0.5, color='tab:purple', label='Delta (norm)')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Действие агента (нормализованное приращение)')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('delta_norm')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['step_interval_us'], alpha=0.7, linewidth=0.5, color='tab:brown')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Интервал между шагами')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Интервал (мкс)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Сравнение PID-exploration vs НС-контроллер

In [ ]:
pid_df = env_df[env_df['global_step'] <= exploration_steps]
nn_df = env_df[env_df['global_step'] > exploration_steps]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(pid_df['error'], bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0].set_title(f'Распределение ошибки: PID (шаги 1–{exploration_steps})')
axes[0].set_xlabel('Error')
axes[0].set_ylabel('Частота')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3)

axes[1].hist(nn_df['error'], bins=100, alpha=0.7, color='tab:orange', edgecolor='black', linewidth=0.3)
axes[1].set_title(f'Распределение ошибки: НС (шаги {exploration_steps + 1}–{env_df["global_step"].max()})')
axes[1].set_xlabel('Error')
axes[1].set_ylabel('Частота')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"PID  — mean |error|: {pid_df['error'].abs().mean():.2f}, std: {pid_df['error'].std():.2f}, median: {pid_df['error'].median():.2f}")
print(f"НС   — mean |error|: {nn_df['error'].abs().mean():.2f}, std: {nn_df['error'].std():.2f}, median: {nn_df['error'].median():.2f}")

## Скользящее среднее ошибки

In [ ]:
alpha = 0.2
env_df['abs_error_ema'] = env_df['error'].abs().ewm(alpha=alpha, adjust=False).mean()

plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['abs_error_ema'], linewidth=1.0, color='tab:red',
         label=f'|Error| (EMA, α={alpha})')
plt.axvline(x=exploration_steps, color='gray', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Экспоненциальное скользящее среднее абсолютной ошибки')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('|Error| (EMA)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ процесса обучения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q1'], alpha=0.6, linewidth=0.3, color='tab:blue', label='Q1 Loss')
plt.title('Critic Q1 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q2'], alpha=0.6, linewidth=0.3, color='tab:green', label='Q2 Loss')
plt.title('Critic Q2 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
actor_df = train_df[train_df['actor_loss'].notna()]

plt.figure(figsize=(14, 5))
plt.plot(actor_df['step'], actor_df['actor_loss'], alpha=0.6, linewidth=0.3, color='tab:red', label='Actor Loss')
plt.title('Actor Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['buffer_size'], linewidth=1.0, color='tab:purple')
plt.title('Размер буфера воспроизведения')
plt.xlabel('Шаг обучения')
plt.ylabel('Размер буфера')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ логов соединения (phase_shifter)

Логи `source=phase_shifter, event=exchange` — сырые обмены с фазовращателем через последовательный порт. Каждая запись содержит отправленный `control_output` и полученный `process_variable`. Эти данные включают все обмены, в том числе во время reset-фаз окружения.

In [ ]:
ps_df = load_jsonl(EXPERIMENT_DIR_PATH / "env.jsonl", source="phase_shifter", event="exchange")
ps_df = ps_df.reset_index(drop=True)
ps_df['global_step'] = ps_df.index + 1
print(f"Обмены с фазовращателем: {len(ps_df)} записей")
print(f"control_output: [{ps_df['control_output'].min()}, {ps_df['control_output'].max()}]")
print(f"process_variable: [{ps_df['process_variable'].min()}, {ps_df['process_variable'].max()}]")
ps_df.head()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(ps_df['global_step'], ps_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
plt.title('Process Variable (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Process Variable')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(ps_df['global_step'], ps_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
plt.title('Control Output (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Control Output')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(ps_df['global_step'], ps_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
axes[0].set_title('Control Output')
axes[0].set_ylabel('Control Output')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ps_df['global_step'], ps_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
axes[1].set_title('Process Variable')
axes[1].set_xlabel('Номер обмена')
axes[1].set_ylabel('Process Variable')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Логи соединения: Control Output и Process Variable', fontsize=13)
plt.tight_layout()
plt.show()